## SciPy

In this notebook in addition to NumPy we'll also be using SciPy. SciPy is a library for (mathematical) functions that are often used in scientific research.
It consists of function for optimization, interpolation, integration, linear algebra, statistics and other topics.
SciPy also has functions to work with sparse matrices.
Since we will work in this notebook with the sparse submodule of the SciPy package we have to import this module separately.

For more information about SciPy visit the following website:
http://docs.scipy.org/doc/scipy/reference/tutorial/index.html

In [1]:
import numpy
import scipy
import scipy.sparse

# 4 Sparse matrices



## Dense versus sparse matrices


Sparseness can refer to either the **content** of a matrix, or to the **representation** of a matrix in the program.

### Content

A dense matrix is a matrix with a lot of nonzero elements.
A sparse matrix is the opposite and contains of a lot of zeros. 

The level of sparsity or density is an indication of how sparse or dense the matrix is.
The sparsity/density can be computed by dividing the number of nonzero elements by the total number of elements.
When this number is low, then the matrix contains a lot of zeros and therefore the matrix is called sparse.
When this number is high, then the matrix contains no or only a few zeros and therefore the matrix is called dense.

### Representation

When a matrix contains only a few nonzero elements then a specialized sparse representation of the matrix can be used. A sparse representation is a represenation in which only the nonzero elements are stored, while the zeros are implicit.

This type of representation can have multiple benefits, especially when matricies become large.
The major advantages of using a sparse matrix representation are:
- Memory: less memory is needed to store the matrix since the zero elements are not stored.
- Efficiency: using a sparse matrix in functions or loops can speed up the process in comparison with using a dense representation of the matrix since the zero elements are skipped in the sparse representation.

### When to use sparse matrix representations in data science: 

When the data content is sparse:

- Matrices containing only diagonal entries (e.g. Eigenvalues, singular value decomposition, other factorization techniques).
- Matricies containing entries only in the upper or lower triangular (e.g. symmetrical similarity values, growth curve information)
- Matricies that code the connections between nodes in a network (e.g. social network analysis, physical networks, Biology) 
- Counting the words in text-based documents. 

## 4.1 Representations of sparse matrix

There are several ways to store sparse matrices in SciPy. Unfortunately there is no sparse matrix representation that can do all tasks efficiently and fast. They generally fall into two groups that specialize in doing one thing really well.

First, there are the group of representations that are efficient to build iteratively and modify individual entries. They are:

- Dictionary of keys (dok_matrix)
- List of lists (lil_matrix)
- Coordinate list (coo_matrix)

Second, there are the group of representations that are effienct to index particular items and perform matrix operations (including linear algebra). They are:

- Compressed sparse column (csc_matrix)
- Compressed sparse row (csr_matrix)

Finally, there are the more specialized group of representations that are most efficient if the entries are non-randomly positioned in certain ways. They are:

- Diagonal storage (dia_matrix)
- Block sparse row matrix (bsr_matrix)

## 4.2 Formats of Sparse matricies

### 4.2.1 Compressed Sparse Row (csr)

CSR represents the matrix with three numpy arrays. They contain 
1. The row indexes of the non-zero entries in the matrix.
2. The column indexes of the non-zero entries in the matrix.
3. All values in the matrix that are non-zero.

Note that the order of the arrays ensures that CSR sparse matricies are sorted by rows, so that all values in the matrix within the same row are consecutive in the sparse matrix. This is not true for entries that are in the same column.

Advantages: 
- Fast row slicing
- Efficient computations that operate across rows

Disadvantages: 
- Slow column slicing operations.
- Changes to the sparsity structure are expensive (use LIL, COO, or DOK instead). 


In [2]:
dense_matrix = numpy.random.randint(4, size=(3,3))

cdr = scipy.sparse.csr_matrix(dense_matrix, shape=dense_matrix.shape)

print( dense_matrix )
print()
print( cdr )

[[0 1 3]
 [0 1 3]
 [0 3 2]]

  (0, 1)	1
  (0, 2)	3
  (1, 1)	1
  (1, 2)	3
  (2, 1)	3
  (2, 2)	2


### 4.2.2 Compressed Sparse Column (csc)

CSC represents the matrix with three numpy arrays. They contain 
1. The column indexes of the non-zero entries in the matrix.
2. The row indexes of the non-zero entries in the matrix.
3. All values in the matrix that are non-zero.

Note that the order of the arrays ensures that CSC sparse matricies are sorted by columns, so that all values in the matrix within the same column are consecutive in the sparse matrix. This is not true for entries that are in the same row.

Advantages: 
- Fast column slicing
- Efficient computations that operate across columns

Disadvantages: 
- Slow row slicing operations.
- Changes to the sparsity structure are expensive. 


In [3]:
csc = scipy.sparse.csc_matrix(dense_matrix, shape=dense_matrix.shape)
print( csc)

  (0, 1)	1
  (1, 1)	1
  (2, 1)	3
  (0, 2)	3
  (1, 2)	3
  (2, 2)	2


### 4.2.3 Dictionary of Keys (dok)

The dictionary of keys sparse matrix representation is a dictionary format where every key in the dictionary represents the row and column indices of the element and the value in the dictionary represents the value of the element at that particular position. Elements with value 0 are not present in the dictionary, but they are assumed to be implicitly zero. 

The Scipy dictionary of keys differs in a few critical ways from dictionaries in vanilla python.
- In scipy the total number of possible rows and columns must be specified at initialization. This can be changed but requires a full copy in memory. 
- Also, the rows and columns in a scipy DOK sparse matrix must be integers. Unlike in vanilla python where keys can be strings, values along either dimension cannot be arbitrary types .

In [4]:
# Create a dok matrix incrementally

S = scipy.sparse.dok_matrix((5, 5), dtype=numpy.int32)
for i in range(5):
    for j in range(5):
        S[i,j] = i*j
print(S)

  (1, 2)	2
  (3, 2)	6
  (1, 3)	3
  (3, 3)	9
  (4, 1)	4
  (3, 1)	3
  (4, 4)	16
  (2, 1)	2
  (2, 4)	8
  (2, 3)	6
  (1, 4)	4
  (4, 3)	12
  (2, 2)	4
  (4, 2)	8
  (3, 4)	12
  (1, 1)	1


## Exercise 4.2

In this exercise we will learn how to build a document-term matrix.
We'll build two functions.

- Create function `word_index`  which takes a list of words, and returns a dictionary mapping each unique word to an index. For example:
```python
word_index("A rose is a rose".split())
```
```
{'A': 0, 'a': 3, 'is': 2, 'rose': 1}
```
- Create function `word_count` which takes a list of texts (where each text is a list of words), and returns two values:
   - a `dok_matrix` which counts how many times each word occurs in each text. The matrix should be $M\times N$, where $M$ is the number of texts and  $N$ is the total number of unique words. 
   - a vanilla python dictionary that maps from words to column indicies in the dok_matrix. This should have $N$ entries.

Example:

```python
text = ['All human beings are born free and equal in dignity and rights'.split(),
       'They are endowed with reason and conscience and should act towards one another in a spirit of brotherhood'.split()]
vocab, M = word_count(text)
print(M[0, vocab['and']])
print(M[1, vocab['reason']])
```       
Output:
```
2.0
1.0
```

In [10]:
def word_index(x):
    return 

In [11]:
# Testing word_index()

print(word_index("A rose is a rose".split()))
print(word_index("I love Data Processing Advanced lectures".split()))
print(word_index("do be do be do".split()))
    

None
None
None


In [12]:
# use this for testing!
text = ['All human beings are born free and equal in dignity and rights'.split(),
       'They are endowed with reason and conscience and should act towards one another in a spirit of brotherhood'.split()]

In [13]:
def word_count(x):
    return 

In [14]:
# test the results!
vocab, M = word_count(text)
print(M[0, vocab['and']])
print(M[1, vocab['reason']])

TypeError: 'NoneType' object is not iterable

## 4.3 Sparse matrix conversion

Since no single sparse matrix representation is best for all types of analysis, it is often more efficient to create one type of sparse matrix and then convert into a different representation. Every sparse matrix representation in Scipy can be converted into any other representation, though this step can often be very computationally expensive and slow.

However, many times you might end up creating a sparse matrix incrementally, using `dok` or `lil` and then converting it to `csc` or `csr` for efficient matrix multiplication or inversion.

All of the details about Scipy sparse matrix representations can be found online:  http://docs.scipy.org/doc/scipy/reference/sparse.html

### 4.3.1 Converting to dense scipy matricies or a numpy array

Some calculations require a dense matrix. Any scipy.sparse matrix can be convered to a scipy.dense matrix (THIS IS NOT A NUMPY array!) or a Numpy array:

- `x.todense()`:  convert sparse matrix to densely represented matrix (not the same as a numpy array!)
- `x.toarray()`: convert sparse matrix into a regular numpy array

In [15]:
x = scipy.sparse.rand(3, 3, density=0.4, format="dok")
print( x )

# transform to dense matrix:
print("Dense matrix: \n", x.todense())

# transform to numpy array:
print("Numpy array: \n", x.toarray())


  (0, 1)	0.721553583297
  (2, 0)	0.301882208207
  (2, 2)	0.706715211385
Dense matrix: 
 [[ 0.          0.72155358  0.        ]
 [ 0.          0.          0.        ]
 [ 0.30188221  0.          0.70671521]]
Numpy array: 
 [[ 0.          0.72155358  0.        ]
 [ 0.          0.          0.        ]
 [ 0.30188221  0.          0.70671521]]


### 4.3.2 Check matrix properties

When you want to check whether or not the matrix is a sparse format then the following function can be helpful:
- `scipy.sparse.issparse(x)`
- `scipy.sparse.isspmatrix(x)`
- `scipy.sparse.isspmatrix_dok(x) / scipy.sparse.isspmatrix_lil(x) / scipy.sparse.isspmatrix_coo(x)`: the function is the same for all different formats and everytime the three letters abbreviations are used. 

All these functions return True in the case where the matrix is sparse and of the particular type and False otherwise.

You can also use the X.getformat() to get the format of a sparse matrix (X).

In [16]:
R = scipy.sparse.rand(5, 5, density=0.2, format="coo", dtype=numpy.float32)

print( "Is sparse?", scipy.sparse.issparse(R) )
print( "Is Dictionary of Keys format?", scipy.sparse.isspmatrix_dok(R) )
print( "Is Coordinate lists format?", scipy.sparse.isspmatrix_coo(R) )
print( "Type of sparse matrix is: ", R.getformat() )

Is sparse? True
Is Dictionary of Keys format? False
Is Coordinate lists format? True
Type of sparse matrix is:  coo


## 4.4 Functions to create a sparse matrix

Scipy.sparse has three functions that create sparse matricies with particular properties. Each of these functions can be passed a parameter `format` to indicate which representational format to create. Three letters abbreviations of the sparse representations are used: `dok, lil, coo, csr, csc, bsr, and dia`. 


In [17]:
# Create a sparse identity matrix
E = scipy.sparse.eye(5, 5, 0, dtype=numpy.int32, format="dok" )
print(E)

  (3, 3)	1
  (0, 0)	1
  (1, 1)	1
  (4, 4)	1
  (2, 2)	1


In [18]:
# Create a sparse random matrix with values from a uniform distribution
R = scipy.sparse.rand(5, 5, density=0.2, format="coo", dtype=numpy.float32)
print(R)

  (1, 3)	0.927421
  (2, 0)	0.97118
  (3, 1)	0.829393
  (3, 0)	0.548979
  (4, 2)	0.85739


In [19]:
# Create a sparse random matrix from an aribrary distribution
from scipy import stats
values = stats.poisson(25, loc=10).rvs

S = scipy.sparse.random(30, 40, density=0.005, data_rvs=values)
print(S)

  (15, 11)	32.0
  (7, 28)	22.0
  (25, 29)	33.0
  (28, 31)	35.0
  (17, 22)	29.0
  (6, 17)	33.0


## 4.5 Using sparse representations

You can use sparse matrix represenations in normal mathematical transformations, such as addition, substraction, division, multiplication, and matrix power. These computations treat the _missing_ values in a sparse matrix as values that are 0. So often these operations result in a matrix that has many more values than the original sparse matrix.

In [20]:
# mathematical transformations with sparse matrices
x = scipy.sparse.rand(4, 4, density=0.5, format="csr", dtype=numpy.float32)
y = scipy.sparse.rand(4, 4, density=0.5, format="csr", dtype=numpy.float32)
print("x=")
print(x)
print("y=")
print(y)
print("x+y=")
print(x+y)
print("x*y")
print(x*y) # this is the dot product!

x=
  (0, 2)	0.664303
  (0, 3)	0.512953
  (1, 0)	0.0546204
  (1, 2)	0.817975
  (2, 1)	0.971419
  (3, 0)	0.280275
  (3, 1)	0.688567
  (3, 3)	0.178408
y=
  (0, 1)	0.783285
  (0, 2)	0.832496
  (0, 3)	0.272266
  (1, 1)	0.0945226
  (1, 3)	0.066404
  (3, 0)	0.531283
  (3, 1)	0.167371
  (3, 3)	0.0787933
x+y=
  (0, 1)	0.783285
  (0, 2)	1.4968
  (0, 3)	0.785219
  (1, 0)	0.0546204
  (1, 1)	0.0945226
  (1, 2)	0.817975
  (1, 3)	0.066404
  (2, 1)	0.971419
  (3, 0)	0.811558
  (3, 1)	0.855937
  (3, 3)	0.257202
x*y
  (0, 3)	0.0404173
  (0, 1)	0.0858534
  (0, 0)	0.272523
  (1, 3)	0.0148713
  (1, 2)	0.0454712
  (1, 1)	0.0427833
  (2, 3)	0.0645061
  (2, 1)	0.091821
  (3, 0)	0.0947853
  (3, 3)	0.13609
  (3, 2)	0.233328
  (3, 1)	0.314481


## 4.6 Often used functions applied to sparse matrices

The major question when using sparse matricies in compuations and functions is will the sparse representation be preserved. Scipy has a number of other useful functions that preserve the sparse representation:

- Calculating the mean of the matrix: `X.mean(axis=0)` can be used to calculate the mean of sparse matrix `X`. This function does not destroy the sparse matrix structure.

- Calculating the minimum or maximum of the matrix without destroying the sparse structure. `x.minimum()` will give the minimum over the matrix. 

Other functions that can be applied to sparse matrices (where x is a sparse matrix):
- `x.transpose()` or `x.T`: return the transpose of the matrix as a sparse matrix.
- `x.sum(axis=)`: calculate the sum over the matrix or the sum over one axis.
- `x.nonzero()`: return the indices of the non-zero elements.
- `x.reshape(shape)`: reshape the array to another shape.
- `x.multiply(y)`: element-wise multiplication of x with another matrix y (be careful with the shape).

Full list: http://docs.scipy.org/doc/scipy/reference/sparse.html

In [21]:
# create a random 10 by 5 sparse matrix with density 0.4 in the dictionary of keys format 
x = scipy.sparse.rand(10, 5, density=0.2, format="dok")

print( x )

# calculate the mean without destroying the sparse structure
print("Mean over the total matrix: ", x.mean())
print("Mean for every column: ", x.mean(axis=0))

# calculate the sum over the matrix
print("Total sum of the matrix:", x.sum())

  (1, 2)	0.170079393408
  (9, 0)	0.156633296229
  (0, 0)	0.884019301449
  (1, 4)	0.960966063284
  (3, 3)	0.198362467611
  (8, 1)	0.921679696658
  (5, 1)	0.982333066849
  (0, 3)	0.948206573725
  (5, 2)	0.317467421357
  (7, 2)	0.239370151012
Mean over the total matrix:  0.115582348632
Mean for every column:  [[ 0.10406526  0.19040128  0.0726917   0.1146569   0.09609661]]
Total sum of the matrix: 5.77911743158


In [22]:
# transpose a sparse matrix
print( "Transpose:" )
print( x.transpose() )

# multiply element-wise
y = scipy.sparse.rand(10, 5, density=0.2, format="dok")
print( "Multiply pointwise:" )
print( x.multiply(y) )

Transpose:
  (2, 7)	0.239370151012
  (1, 5)	0.982333066849
  (1, 8)	0.921679696658
  (0, 0)	0.884019301449
  (3, 3)	0.198362467611
  (0, 9)	0.156633296229
  (2, 5)	0.317467421357
  (4, 1)	0.960966063284
  (3, 0)	0.948206573725
  (2, 1)	0.170079393408
Multiply pointwise:
  (0, 0)	0.739298220389
  (7, 2)	0.179705992924


### 4.6.1 Warning: Sparse matrix dot product 

When using sparse matrices you should not use the code: `numpy.dot(x,y)` to get the dot product between a matrix x and a vector y. Use `x.dot(y)` instead.

Sparse matrices are not supported in `numpy.dot(x,y)` and therefore this can lead to errors. 
Reference: http://docs.scipy.org/doc/scipy/reference/sparse.html#matrix-vector-product


In [23]:
x = scipy.sparse.dok_matrix([[1, 2, 0], [0, 0, 3], [4, 0, 5]])
y = numpy.array([5, 6, 0])

# the Scipy dot product function works for Scipy sparse matricies
print("Scipy version:")
print(x.dot(y))
print()

# the Numpy dot product function does not work for Scipy sparse matricies
print("Numpy version:")
print(numpy.dot(x, y))

Scipy version:
[17  0 20]

Numpy version:
[ <3x3 sparse matrix of type '<class 'numpy.int64'>'
	with 5 stored elements in Dictionary Of Keys format>
 <3x3 sparse matrix of type '<class 'numpy.int64'>'
	with 5 stored elements in Dictionary Of Keys format>
 <3x3 sparse matrix of type '<class 'numpy.int64'>'
	with 5 stored elements in Dictionary Of Keys format>]


## 4.6.2 Adding or subtracting a scalar

Adding or subtracting a value from all values of a sparse matrix tends to have very unexpected behavior. In effect, this changes the matrix so that most values are not zero. The matrix representation will remain sparse, but most of the content of the matrix will not be zero and thus not sparse.

The same warning holds for any normalization or standardization of a sparse matrix.

In [24]:
x = scipy.sparse.rand(3, 3, density=0.4, format="dok")

print( x )

# allows you to add a scalar value to all values in a sparse matrix:
print("So many values!:")
print(x + 2 * x.mean())

# but prevents you from doing subtraction because it is not implimented
print(x-x.mean())

  (0, 1)	0.513197147741
  (0, 0)	0.642664707672
  (2, 2)	0.00170471865191
So many values!:
  (0, 1)	0.7704341642
  (1, 2)	0.257237016459
  (0, 0)	0.899901724131
  (2, 0)	0.257237016459
  (1, 0)	0.257237016459
  (2, 2)	0.258941735111
  (0, 2)	0.257237016459
  (2, 1)	0.257237016459
  (1, 1)	0.257237016459


NotImplementedError: subtracting a nonzero scalar from a sparse matrix is not supported

## Exercise 4.6.1

Given a word index dictionary and a document-term matrix returned by `word_count`:

Part A: find the most frequent word for each document.

Part B: find the document with the most words

## Exercise 4.6.2

Function `f` takes a document-term matrix, and returns another matrix. Analyze the code in `f` and try to figure out what this function is doing and how to interpret the result it returns.
```python
def f(D):
    M = D.minimum(1.0)
    return M.dot(M.T)
```

## Exercise 4.6.3

- Create a list of texts by reading the first 1000 lines from [coco_val.txt](coco_val.txt). Create matrix A with the document-term matrix from these 1000 sentences. Create another matrix B by converting the matrix A from `dok` format to `csr` format. Use the ipython command `%timeit` to compare whether it's faster to apply function `f` (see above) to matrix A or B.
- Create matrix C by converting A to a regular numpy array. Write function `g` which is like `f`, but works on regular numpy array arrays. Use `%timeit` to see how fast g is when applied to matrix C.

## Exercise 4.12

There are a number of ways of measuring similarity between documents. For this exercise we'll try to use the number of unique words the documents have in common (word overlap).


1. Define function `similarity` which takes a document-term matrix, and returns a square matrix where the value in the row $i$ and column $j$ is the word overlap between the $i$th and $j$th document.
3. Calculate the overlap similarity matrix for sentences in [coco_val.txt](coco_val.txt). For the first 20 sentences, display the 3 most similar sentences according to this matrix.
4. Word overlap is a very simplistic similarity measure. Suggest, and possibly implement, a better way of measuring similarity between documents based on a document-term matrix.